In [3]:
%matplotlib inline
import pandas as pd
import matplotlib
import numpy as np
import random
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import seaborn as sns; sns.set()
from collections import namedtuple
np.core.arrayprint._line_width = 160

#### Random orthonormal vectors

In [ ]:
vecs = np.matrix(bm.randOrthoNormals(d=5, count=2))
(vecs, np.transpose(vecs) * vecs)

In [ ]:
bm.randOrthoNormals(d=5, count=2)

#### Create tuples that label lattice points

In [ ]:
lat.LatticeTuplesD1(None, x_len=2), lat.LatticeTuplesD1("a", x_len=2)

In [ ]:
lat.LatticeTuplesD2(None, x_len=2, y_len=2), lat.LatticeTuplesD2("a", x_len=2, y_len=1)

In [ ]:
lat.LatticeTuplesD3(None, x_len=2, y_len=1, z_len=2), lat.LatticeTuplesD3("a", x_len=2, y_len=1, z_len=1)

#### some constants that are used in the next few sections

In [ ]:
dim = 2; stepsX = 3; stepsY = 4; stepsZ = 1

In [ ]:
[(x, y) for x in range(stepsX) for y in range(stepsY)]

#### For creating 2d numpy arrays of uniform lattices over the unit N-cube for N = {2, 3}

In [ ]:
lat.MakeD2UnitLattice(span_x=stepsX, span_y=stepsY)

In [ ]:
lat.MakeD3UnitLattice(span_x=stepsX, span_y=stepsY, span_z=stepsZ)

#### Color mapping

In [ ]:
cr = lp.MakeRedBlueGridColorer(span_x=stepsX, span_y=stepsY)
clrs = [cr((x, y)) for x in range(stepsX) for y in range(stepsY)]
sns.palplot(clrs)

#### Create a 2d lattice of vectors in N dimenstional space, packaged as TaggedNVectors

In [ ]:
rasty = lat.MakeD2Lattice(spans=np.identity(dim), steps1=stepsX, steps2=stepsY, center=np.array([1, 10]))
r = lp.PlotTaggedNVectorsGrid(rasty, span_x=stepsX, span_y=stepsY, figsize=(6, 4), markersize=10)

In [ ]:
q = lat.MakeRndOrthoD2Lattice(dim=dim, steps1=stepsX, steps2=stepsY, seed=1234)
outs = lp.PlotTaggedNVectorsGrid(q, span_x=stepsX, span_y=stepsY, figsize=(6, 4), markersize=10)

#### Create a set of uniform gaussians around each of the provided vectors

In [ ]:
def drawAroundVecs(tvs:dt.TaggedNVectors, blobStdev:float=1.0, pointsPerBlob:int=10):
    blobVecs = tvs.vectors
    blobTags = tvs.tags
    dim = len(blobVecs[0])
    return dt.TaggedNVectors(
        vectors = np.vstack(np.array([np.random.multivariate_normal(
                                        blobVecs[i], 
                                        np.matrix(np.diag([blobStdev] * dim)), 
                                        pointsPerBlob) 
                              for i in range(len(blobVecs))])),
        tags = [i for i in blobTags for a in range(pointsPerBlob)]
    )

In [ ]:
testCenters = dt.TaggedNVectors(vectors=np.array([[0,0], [1,1]]), tags=[(0,1), (1,1)])
blobs = drawAroundVecs(tvs=testCenters, blobStdev=0.25, pointsPerBlob=50)

rr = lp.PlotTaggedNVectors(tvs=blobs, markersize=5)

#### Create a 2d lattice of gaussians in the plane specified by the two input vectors.

In [ ]:
def GetUnitMatrix(dim:int, s:float):
    '''returns a diagonal matrix with all s on the diagonal'''
    return np.identity(dim) * s

In [ ]:
q = GetUnitMatrix(3, 33)
q * 3


In [ ]:
quak = np.matrix([[1,2], [3,4]])
expp = np.matrix(np.diag([2,2]))
type(quak * expp)

In [ ]:
rasty = lat.MakeD2Lattice(spans=np.identity(dim), steps1=stepsX, steps2=stepsY)
def d2LatticeOfGaussians(spans:np.matrixlib.defmatrix.matrix=np.identity(2), arrayRadius:float=1, steps1:int=3, steps2:int=4, blobStdev:float=1.0, pointsPerBlob:int=10):
    d2Lattice = rasterOverCenteredSquare(v1=v1, v2=v2, radius=arrayRadius, stride=stride)
    return drawAroundVecs(tvs=TaggedNVectors(vectors=d2Lattice.vectors, tags=d2Lattice.coords), 
                          blobStdev=blobStdev, 
                          pointsPerBlob=pointsPerBlob)

In [ ]:
blobStride = 4
blobsL = d2LatticeOfGaussians(
                          arrayRadius=3, 
                          steps1=blobStride, 
                          steps2=blobStride, 
                          blobStdev=0.01, 
                          pointsPerBlob=50)
r4 = PlotTaggedNVectorsGrid(blobsL, span_x=blobStride, span_y=blobStride, figsize=(6,6), markersize=5)

#### Samples from the N dimensional symmetric gaussian.

#### Nested lattices

In [ ]:
dim = 5
count = 10
mean = [np.random.randn() for i in range(dim)]
np.random.multivariate_normal(mean, np.matrix(np.diag([1] * dim)), count)

In [ ]:
import sklearn
from sklearn.manifold import TSNE

### Stride 7, pointsPerBlob=100, dim = 100, arrayRadius=3 - vary seed, stdev

In [ ]:
strideA = 7
ppb = 100
dim = 100
arrayRadius=3
def varyStdevA(stdev, seed):
    orthoPair = randOrthoNormals(dim=dim, count=2)
    gridBlobs = d2LatticeOfGaussians(v1=orthoPair[0],
                          v2=orthoPair[1],
                          arrayRadius=arrayRadius,
                          stride=strideA,
                          blobStdev=stdev,
                          pointsPerBlob=ppb)
    grid_proj = TSNE(n_components=2, random_state=seed, init='random').fit_transform(gridBlobs.vectors)
        
    tsnu = TaggedNVectors(vectors=grid_proj, tags=gridBlobs.tags)
    r77 = PlotTaggedNVectorsGrid(tsnu, span_x=strideA, span_y=strideA, figsize=(6,6), markersize=3)

In [ ]:
varyStdevA(0.075, 98154190)

In [ ]:
varyStdevA(0.1, 98154190)

In [ ]:
varyStdevA(0.15, 82154190)

### Stride 7, pointsPerBlob=100, dim = 100, arrayRadius=4 - vary stdev, seed

In [ ]:
strideA = 7
ppb = 100
dim = 100
arrayRadius=4
def varyStdevB(stdev, seed):
    orthoPair = randOrthoNormals(dim=dim, count=2)
    gridBlobs = d2LatticeOfGaussians(v1=orthoPair[0],
                          v2=orthoPair[1],
                          arrayRadius=arrayRadius,
                          stride=strideA,
                          blobStdev=stdev,
                          pointsPerBlob=ppb)
    grid_proj = TSNE(n_components=2, random_state=seed, init='random').fit_transform(gridBlobs.vectors)
        
    tsnu = TaggedNVectors(vectors=grid_proj, tags=gridBlobs.tags)
    r77 = PlotTaggedNVectorsGrid(tsnu, span_x=strideA, span_y=strideA, figsize=(6,6), markersize=3)

In [ ]:
varyStdevB(0.075, 98154190)

In [ ]:
varyStdevB(0.1, 98154190)

In [ ]:
varyStdevB(0.15, 98154190)

### Stride=7, pointsPerBlob=200, dim=100, arrayRadius=4 - vary stdev, seed

In [ ]:
strideA = 7
ppb = 200
dim = 100
arrayRadius=4
def varyStdevC(stdev, seed):
    orthoPair = randOrthoNormals(dim=dim, count=2)
    gridBlobs = d2LatticeOfGaussians(v1=orthoPair[0], 
                          v2=orthoPair[1], 
                          arrayRadius=arrayRadius, 
                          stride=strideA, 
                          blobStdev=stdev, 
                          pointsPerBlob=ppb)
    grid_proj = TSNE(n_components=2, random_state=seed, init='random').fit_transform(gridBlobs.vectors)
        
    tsnu = TaggedNVectors(vectors=grid_proj, tags=gridBlobs.tags)
    r77 = PlotTaggedNVectorsGrid(tsnu, span_x=strideA, span_y=strideA, figsize=(6,6), markersize=3)

In [ ]:
varyStdevC(0.075, 98154190)

In [ ]:
varyStdevC(0.1, 98154190)

In [ ]:
varyStdevC(0.15, 98154190)

### Stride=6, pointsPerBlob=200, dim = 100, arrayRadius=5 - vary stdev, seed

In [ ]:
strideA = 6
ppb = 200
dim = 100
arrayRadius=5
def varyStdevD(stdev, seed):
    orthoPair = randOrthoNormals(dim=dim, count=2)
    gridBlobs = d2LatticeOfGaussians(v1=orthoPair[0], 
                          v2=orthoPair[1], 
                          arrayRadius=arrayRadius, 
                          stride=strideA, 
                          blobStdev=stdev, 
                          pointsPerBlob=ppb)
    grid_proj = TSNE(n_components=2, random_state=seed, init='random').fit_transform(gridBlobs.vectors)
        
    tsnu = TaggedNVectors(vectors=grid_proj, tags=gridBlobs.tags)
    r77 = PlotTaggedNVectorsGrid(tsnu, span_x=strideA, span_y=strideA, figsize=(6,6), markersize=3)

In [ ]:
varyStdevD(0.1, 98154190)

In [ ]:
varyStdevD(0.15, 98154190)

In [ ]:
varyStdevD(0.2, 98154190)

In [ ]:
strideA = 2
ppb = 2
dim = 3
arrayRadius=10
orthoPair = randOrthoNormals(dim=dim, count=2)
gridBlobs = d2LatticeOfGaussians(v1=orthoPair[0], 
                      v2=orthoPair[1], 
                      arrayRadius=arrayRadius, 
                      stride=strideA, 
                      blobStdev=0.001, 
                      pointsPerBlob=ppb)

In [ ]:
gridBlobs

In [ ]:
tup = (1, 2)
(tup, 1, 2)